### data

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [2]:
alpha = pd.read_csv("result_alpha.csv").to_numpy().flatten()
beta_plus = pd.read_csv("result_beta_plus.csv").to_numpy()
beta_minus = pd.read_csv("result_beta_minus.csv").to_numpy()

R_flat = pd.read_csv("result_R.csv")

I_len = R_flat["generator"].nunique()
T_len = R_flat["time"].nunique() 
S_len = R_flat["scenario"].nunique()

R = R_flat.pivot_table(
    values="value",
    index="generator",
    columns=["time", "scenario"]
).to_numpy()
R = R.reshape(I_len, T_len, S_len)

I, T, S = R.shape

In [3]:
price_q = pd.read_csv(
    "/Users/jangseohyun/Documents/workspace/symply/DER/DATA_price.csv"
)
price_q["Time"] = pd.to_datetime(price_q["Time"], format="%Y-%m-%d %H:%M")

price_q["Hour"] = price_q["Time"].dt.floor("h")
price_h = price_q.groupby("Hour").mean(numeric_only=True)

price = price_h.iloc[: S * T]

P_DA = np.array(
    [sum(price["Price"].iloc[t + s * T] for s in range(S)) / S * 1.2 for t in range(T)]
)
P_RT = np.array([[price["Price"].iloc[t + s * T] for s in range(S)] for t in range(T)])
P_PN = np.array(
    [sum(price["Price"].iloc[t + s * T] for s in range(S)) / S * 2 for t in range(T)]
)

### settlement model

In [4]:
set = gp.Model("Settlement")
set.Params.OutputFlag = 0
set.setParam('TimeLimit', 3600)
set.setParam('MIPGap', 0.01)
set.setParam('Presolve', 0)

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z_y = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
e_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
e_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")

set.update()

Set parameter Username
Set parameter LicenseID to value 2611964
Academic license - for non-commercial use only - expires 2026-01-20


In [5]:
obj = gp.quicksum(
    P_DA[t] * x[i, t] for i in range(I) for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * e_plus[i, t, s] - P_PN[t] * e_minus[i, t, s])
    for i in range(I) for t in range(T) for s in range(S)
)
set.setObjective(obj, GRB.MAXIMIZE)

In [6]:
# 기존 제약식

# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

# real-time market 참여 제약
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i,t,s] - y_plus[i, t, s] >= 0)

# y+, y- 둘 중 하나는 0이어야함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= M * z_y[i, t, s])
            set.addConstr(y_minus[i, t, s] <= M * (1 - z_y[i, t, s]))

# alpha는 모든 x의 합
for t in range(T):
    set.addConstr(
        gp.quicksum(x[i, t] for i in range(I)) == alpha[t]
    )

# 초과량이 있는 발전기만 다른 발전기에게 전력을 줄 수 있음
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= M * z_y[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)

# β+, β- 매칭
for t in range(T):
    for s in range(S):
        # 실제 남는 양(초과량 - 주는 양)의 합이 beta+와 같아야 함
        set.addConstr(
            gp.quicksum(
                e_plus[i, t, s]
                for i in range(I)
            )
            == beta_plus[t, s]
        )
        # 실제 부족한 양(부족량 - 받는 양)의 합이 beta-와 같아야 함
        set.addConstr(
            gp.quicksum(
                e_minus[i, t, s]
                for i in range(I)
            )
            == beta_minus[t, s]
        )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 발전기가 주는 총량은 자신의 초과량을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= y_plus[i, t, s]
            )
            # 발전기가 받는 총량은 자신의 부족량을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= y_minus[i, t, s]
            )

In [7]:
# 거래 후 남은 양 정의
for i in range(I):
    for t in range(T):
        for s in range(S):
            # e_plus: 초과량에서 준 양을 뺀 것
            set.addConstr(
                e_plus[i,t,s] == y_plus[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i)
            )
            # e_minus: 부족량에서 받은 양을 뺀 것
            set.addConstr(
                e_minus[i,t,s] == y_minus[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i)
            )

# # x값 균등 분배 제약
# for t in range(T):
#     for i in range(I):
#         set.addConstr(x[i,t] <= alpha[t]/I)

In [8]:
set.optimize()

if set.status == GRB.OPTIMAL:
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array(
        [[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    y_minus_vals = np.array(
        [[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    d_vals = np.array(
        [
            [[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)]
            for i in range(I)
        ]
    )
    e_plus_vals = np.array(
        [[[e_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    e_minus_vals = np.array(
        [[[e_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    print("Optimal solution found!")
    print(set.objVal)
else:
    print("\nNo optimal solution found.")

Optimal solution found!
15918.10329532185


In [9]:
print(f"Status: {set.status}")
if set.status == GRB.OPTIMAL:
    print("모델이 Optimal입니다")
elif set.status == GRB.UNBOUNDED:
    print("모델이 Unbounded입니다")
    # Unbounded ray 확인
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INFEASIBLE:
    print("모델이 Infeasible입니다")
    # IIS 계산
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INF_OR_UNBD:
    print("모델이 Infeasible 또는 Unbounded입니다")
    # Presolve를 끄고 다시 시도
    set.setParam('Presolve', 0)
    set.optimize()
elif set.status == GRB.TIME_LIMIT:
    print("시간 제한에 도달했습니다")
    if set.SolCount > 0:
        print(f"현재까지의 최선해: {set.objVal}")

Status: 2
모델이 Optimal입니다


### 결과 분석

In [10]:
for t in range(13,15):
    x_sum = sum(x_vals[i, t] for i in range(I))

    for s in range(S):
        y_plus_sum = sum(y_plus_vals[i, t, s] for i in range(I))
        y_minus_sum = sum(y_minus_vals[i, t, s] for i in range(I))

        e_plus_sum = sum(e_plus_vals[i, t, s] for i in range(I))
        e_minus_sum = sum(e_minus_vals[i, t, s] for i in range(I))

        print(f"[t={t} s={s}]")

        print(f"alpha: {alpha[t]:.2f}")

        print(f"x 합계: {x_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {x_vals[i,t]:.2f}", end=" ")
        print(")")

        print(f"y+ 합계: {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"y- 합계: {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e+ 합계: {e_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e- 합계: {e_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"beta+: {beta_plus[t,s]:.2f}")
        print(f"beta-: {beta_minus[t,s]:.2f}")

        print("거래 내역:")
        for i in range(I):
            for j in range(I):
                if d_vals[i, j, t, s] > 1e-6:
                    print(
                        f"발전기 {i}가 발전기 {j}에게 {d_vals[i,j,t,s]:.2f} 전력을 줌"
                    )
        print()

[t=13 s=0]
alpha: 70.54
x 합계: 70.54 ([0] 3.71 [1] 3.71 [2] 3.71 [3] 3.71 [4] 3.71 [5] 3.71 [6] 3.71 [7] 3.71 [8] 3.71 [9] 3.71 [10] 3.71 [11] 3.71 [12] 3.71 [13] 3.71 [14] 3.71 [15] 3.71 [16] 3.71 [17] 3.71 [18] 3.71 )
y+ 합계: 41.69 ([0] 0.00 [1] 0.00 [2] 4.11 [3] 0.00 [4] 0.00 [5] 6.32 [6] 1.13 [7] 2.75 [8] 0.33 [9] 4.36 [10] 5.82 [11] 0.00 [12] 6.48 [13] 2.65 [14] 0.00 [15] 3.71 [16] 0.00 [17] 4.03 [18] 0.00 )
y- 합계: 18.57 ([0] 3.68 [1] 1.33 [2] 0.00 [3] 1.82 [4] 0.08 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 [10] 0.00 [11] 3.71 [12] 0.00 [13] 0.00 [14] 1.72 [15] 0.00 [16] 3.71 [17] 0.00 [18] 2.50 )
e+ 합계: 23.12 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 1.13 [7] 2.75 [8] 0.33 [9] 0.00 [10] 5.82 [11] 0.00 [12] 6.48 [13] 2.65 [14] 0.00 [15] 3.71 [16] 0.00 [17] 0.26 [18] 0.00 )
e- 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 [10] 0.00 [11] 0.00 [12] 0.00 [13] 0.00 [14] 0.00 [15] 0.00 [16] 0.00 [17] 0.00 [18] 0.00 

In [11]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in range(I):
    for t in range(T):
        total_da_profit_obj += P_DA[t] * x[i,t].x

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT[t, s] * e_plus[i, t, s].x
            total_rt_profit_obj += 1/S * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * e_minus[i, t, s].x
            total_penalty_cost_obj += 1/S * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"총 Day-ahead 수익 (_obj): {total_da_profit_obj:.2f}")
print(f"총 Real-time 수익 (_obj): {total_rt_profit_obj:.2f}")
print(f"총 Penalty 비용 (_obj): {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

총 Day-ahead 수익 (_obj): 13898.50
총 Real-time 수익 (_obj): 2420.42
총 Penalty 비용 (_obj): 400.82
목적 함수 기반 총 이익 (_obj): 15918.10


In [12]:
# 모든 der의 profit 합계 계산
total_der_profit = 0
der_profit = {}
for i in range(I):
    # 각 der i의 profit 합계 계산
    der_profit[i] = sum(P_DA[t] * x[i,t].x + sum(1/S * (P_RT[t,s] * e_plus[i,t,s].x - P_PN[t] * e_minus[i,t,s].x) for s in range(S)) for t in range(T))
    total_der_profit += der_profit[i]

print("\nder_profit:")
for i in range(I):
    print(f"{[i]} {der_profit[i]:.2f}")

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")



der_profit:
[0] 884.65
[1] 862.93
[2] 853.65
[3] 875.97
[4] 855.86
[5] 776.35
[6] 941.93
[7] 997.49
[8] 818.14
[9] 868.92
[10] 866.91
[11] 707.93
[12] 955.22
[13] 929.98
[14] 782.69
[15] 746.55
[16] 688.40
[17] 744.88
[18] 759.66

모든 der의 profit 합계: 15918.10
